## 准备输入和输出目录

In [1]:
test_data_dir = '/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/origin_data/test_data

In [2]:
import os
outdir = '/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout'
dataname = 'test'
stockdir = outdir + '/stockdir'
usagedir = outdir + '/usagedir'
parquet_dir = os.path.join(stockdir, dataname, 'parquet_dir')
pack_dir = os.path.join(usagedir, dataname, 'pack_dir')
info_outdir = os.path.join(stockdir, dataname, 'info_dir')

In [3]:
reout = '/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/reout'
re_paruquet = os.path.join(reout, 'reparquet_dir')
re_pack = os.path.join(reout, 'repack_dir')

## 数据打包

In [3]:
import dodrio 

In [4]:
dodrio.gen_parquet(test_data_dir, parquet_dir, mid_name=dataname, file_type='wav')

Turn0LoadAudio: 100%|██████████| 373/373 [00:00<00:00, 2402.21it/s]


/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/stockdir/test/parquet_dir/wav_test_00000.parquet had be saved


In [5]:
dodrio.gen_package(test_data_dir, pack_dir, mid_name=dataname, target_sample_rate=48000, file_type='wav')

wav_test_00000.pack Processing: 100%|██████████| 373/373 [00:00<00:00, 1702.64it/s]


In [5]:
dodrio.parquet2package(parquet_dir, pack_dir, sample_rate=48000)

wav_test_00000 Processing: 100%|██████████| 373/373 [00:03<00:00, 111.04it/s]


## 打包文件还原音频

In [8]:
dodrio.parquet2wav(parquet_dir, re_paruquet)

wav_test_00000 Processing: 100%|██████████| 373/373 [00:00<00:00, 1287.33it/s]


In [5]:
dodrio.package2wav(pack_dir, re_pack)

100%|██████████| 373/373 [00:00<00:00, 1573.77it/s]


## 文本等info信息保存

In [6]:
info_type = 'libritts'
info_outdir = os.path.join(stockdir, dataname, 'info_dir')
dodrio.gen_infodir(parquet_dir, test_data_dir, info_outdir, info_type, kl=['text', 'unnorm_text'], lang='en', from_type='parquet')

LoadInfo4Supdir: 100%|██████████| 4/4 [00:00<00:00, 164.16it/s]

wav_test_00000.info had been Saved


In [5]:
info_type = 'libritts'
pack_info_outdir = os.path.join(usagedir, dataname, 'info_dir')
dodrio.gen_infodir(pack_dir, test_data_dir, pack_info_outdir, info_type, kl=['text', 'unnorm_text'], lang='en', from_type='pack')

LoadInfo4Supdir: 100%|██████████| 4/4 [00:00<00:00, 160.64it/s]

wav_test_00000.info had been Saved


In [6]:
supdir_list = [usagedir+'/'+dataname]
listoutdir = '/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/listoutdir'

In [7]:
dodrio.genListDir(supdir_list, listoutdir, prefix='test', subnum=50000)

## 特征提取样例

In [8]:
from dodrio.afeat.exp_fun import extractor_embedding

In [9]:
tt = extractor_embedding('/home/jovyan/chenyixiang/workspace/20241024_cosyvoice_flow/online_use/CosyVoice-300M-yk-250122-small-mixspk/campplus.onnx')

In [10]:
extractor_func = tt.extractor

In [11]:
from_type = 'package'
featname = 'embed'
input_dir = pack_dir
out_dir = os.path.join(usagedir, dataname, featname+'_dir')

In [12]:
utt2spk = dodrio.get_utt2spk(info_outdir)

In [13]:
dodrio.extract_feat(extractor_func, featname, input_dir, out_dir, from_type, utt2spk=utt2spk)

wav_test_00000 Processing: 100%|██████████| 373/373 [00:39<00:00,  9.40it/s]


In [14]:
tt.mean_spk_embedding()

In [15]:
featname = 'spkembed'
input_dir = pack_dir
out_dir = os.path.join(usagedir, dataname, featname+'_dir')
extractor_func = tt.spk_embedding_save
dodrio.extract_feat(extractor_func, featname, input_dir, out_dir, from_type, utt2spk=utt2spk)

wav_test_00000 Processing: 100%|██████████| 373/373 [00:00<00:00, 56515.98it/s]


In [12]:
out_dir

'/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/stockdir/test/spkembed_dir'

In [5]:
from dodrio.afeat.exp_fun import speech_token_extractor
tt = speech_token_extractor('/home/jovyan/chenyixiang/workspace/20241024_cosyvoice_flow/online_use/CosyVoice-300M-yk-250122-small-mixspk/speech_tokenizer_v1.onnx')

2025-03-27 11:26:50.306409829 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-03-27 11:26:50.306442480 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [6]:
extractor_func = tt.extractor
from_type = 'package'
featname = 'speechtoken'
input_dir = pack_dir
out_dir = os.path.join(usagedir, dataname, featname+'_dir')

In [7]:
dodrio.extract_feat(extractor_func, featname, input_dir, out_dir, from_type)

wav_test_00000 Processing: 100%|██████████| 373/373 [00:11<00:00, 31.15it/s]


## 其他数据特征

In [ ]:
from dodrio.afeat.exp_audiof import extract_mfe
extractor_func = extract_mfe
from_type = 'package'
featname_list = ['pitch', 'energy', 'mel']
input_dir = pack_dir
out_dir_list = [os.path.join(usagedir, dataname, 'pitch_dir'), os.path.join(usagedir, dataname, 'energy_dir'), os.path.join(usagedir, dataname, 'mel_dir')]

In [ ]:
dodrio.extract_feat_multi(extractor_func, featname_list, input_dir, out_dir_list, from_type, f0etype='dio')

In [ ]:
from dodrio.afeat.exp_load import get_mfa
extractor_func = get_mfa
from_type = 'package'
featname = 'align'
input_dir = pack_dir
out_dir = os.path.join(usagedir, dataname, featname+'_dir')
jsondir = test_data_dir

In [ ]:
dodrio.extract_feat_align(extractor_func, featname, input_dir, out_dir, from_type, jsondir)

In [ ]:
from dodrio.afeat.exp_fun import emotion2vec_extractor
tt = emotion2vec_extractor()
extractor_func = tt.extractor
from_type = 'package'
featname = 'emotionembed'
input_dir = pack_dir
out_dir = os.path.join(usagedir, dataname, featname+'_dir')

In [ ]:
dodrio.extract_feat_extrainfo(extractor_func, featname, input_dir, out_dir, from_type)

## 数据表生成样例

In [5]:
supdir_list = [os.path.join(usagedir, dataname)]
listoutdir= '/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/listoutdir'
featlist= ['embed', 'spkembed', 'speechtoken']

In [6]:
dodrio.gen_datalist(supdir_list, listoutdir, featlist, dodrio.check_func, prefix='test')

### 另一种表格

In [ ]:
supdir_list = [os.path.join(usagedir, dataname)]
listoutdir= '/aliyun-oss/tts_data/dodrio_data/data_list/gstest'
featlist= ['mel', 'pitch', 'energy', 'emotionembed', 'align']

In [ ]:
dodrio.gen_datalist(supdir_list, listoutdir, featlist, dodrio.check_func, prefix='test')

In [ ]:
from dodrio.afeat.exp_audiof import saveStMag
stinpfile = listoutdir + '/all_usage_utt.list'
energy_outfile = listoutdir + '/energy_stmag'
pitch_outfile = listoutdir + '/pitch_stmag'

In [ ]:
saveStMag(stinpfile, pitch_outfile, 'pitch')
saveStMag(stinpfile, energy_outfile, 'energy')

## 单条数据加载样例

In [1]:
infoline = '296_142727_000010_000000|/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/usagedir/test/pack_dir/wav_test_00000.pack|119946232|121158716|4|This reduction, if admitted, would much facilitate the introduction of emotion into our system, which, being founded on the distinction between the consciousness and the object, is likewise an intellectualist system.|en|embed|/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/usagedir/test/embed_dir/wav_test_00000.embed|135168|135936|192|spkembed|/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/usagedir/test/spkembed_dir/wav_test_00000.spkembed|135168|135936|192|speechtoken|/home/jovyan/chenyixiang/workspace/20250324_dodrio/testdata/testout/usagedir/test/speechtoken_dir/wav_test_00000.speechtoken|250040|252568|632'

In [4]:
data_dict = dodrio.load_data_from_line(infoline)

In [5]:
data_dict.keys()

dict_keys(['uttid', 'audio', 'spkid', 'text', 'language', 'embed', 'spkembed', 'speechtoken'])

In [6]:
data_dict

{'uttid': '296_142727_000010_000000',
 'audio': array([ 0.00000000e+00,  0.00000000e+00,  3.05175781e-05, ...,
        -3.35693359e-04, -3.35693359e-04, -2.13623047e-04]),
 'spkid': '4',
 'text': 'This reduction, if admitted, would much facilitate the introduction of emotion into our system, which, being founded on the distinction between the consciousness and the object, is likewise an intellectualist system.',
 'language': 'en',
 'embed': array([-0.7374816 ,  1.7644295 ,  1.2225451 , -0.04643741,  0.73244995,
        -0.7787548 , -0.30873984, -1.3703386 , -0.24599129,  0.8653064 ,
        -0.03411579, -0.41940162,  0.54753184, -2.1319628 , -1.0698097 ,
        -0.767238  ,  0.7527503 ,  0.32521054, -0.40862432, -0.48393655,
         0.31998435, -0.5681067 , -0.2098108 , -1.2162963 , -0.01822931,
         0.12243897, -1.119927  ,  2.259004  ,  0.06196146,  2.182487  ,
         0.57387555, -0.8731336 , -0.16493857,  2.150069  , -0.44916284,
        -1.121672  , -1.1266332 , -0.8603981 

### 带align的加载方法

In [ ]:
from dodrio import load_data_from_line_align

In [ ]:
aa = "shaoniangexing_00002537|/aliyun-oss/tts_data/dodrio_data/data_save/usagedir/dia600h/pack_dir/wav_dia600h_00269.pack|153255360|153596160|3|我劝你还是让开的好一起上|ZH|sil w o3 q van4 n i3 h ai2 sh ir4 r ang4 k ai1 d e5 h ao3 sil y i4 q i3 sh ang4 sil|340 160 10 120 90 60 50 70 50 90 60 50 120 70 50 30 40 90 330 630 90 40 70 50 80 180 530|mel|/aliyun-oss/tts_data/dodrio_data/data_save/usagedir/dia600h/mel_dir/wav_dia600h_00269.mel|47824000|47930240|80,332|pitch|/aliyun-oss/tts_data/dodrio_data/data_save/usagedir/dia600h/pitch_dir/wav_dia600h_00269.pitch|597800|599128|332|energy|/aliyun-oss/tts_data/dodrio_data/data_save/usagedir/dia600h/energy_dir/wav_dia600h_00269.energy|597800|599128|332"


In [ ]:
dd = load_data_from_line_align(aa)